In [1]:
from gcn import TwoLayerGCN
from gcn_edge import TwoLayerGCNEdge

from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
data_name = 'Pubmed'

### Node classification with GCN

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset = Planetoid('./data', data_name, transform=T.NormalizeFeatures())
data = dataset[0]

In [5]:
num_nodes, num_features = data.x.size()
num_classes = len(data.y.unique())

In [6]:
X = data.x
y = F.one_hot(data.y)

# separate train, val, test
X_tr, X_val, X_te = X[data.train_mask], X[data.val_mask], X[data.test_mask]
y_tr, y_val, y_te = y[data.train_mask], y[data.val_mask], y[data.test_mask]

# get indices of train, val, test
mask_tr = torch.cat([torch.ones(len(X_tr)), torch.zeros(num_nodes - len(X_tr))]).bool()
mask_val = torch.cat([torch.ones(len(X_val)), torch.zeros(num_nodes - len(X_val))]).bool()
mask_te = torch.cat([torch.ones(len(X_te)), torch.zeros(num_nodes - len(X_te))]).bool()

# fix shapes
X_tr = torch.cat([X_tr, torch.zeros(size=(num_nodes - X_tr.shape[0], num_features))])
y_tr = torch.cat([y_tr, -torch.ones(size=(num_nodes - y_tr.shape[0], num_classes))])

X_val = torch.cat([X_val, torch.zeros(size=(num_nodes - X_val.shape[0], num_features))])
y_val = torch.cat([y_val, -torch.ones(size=(num_nodes - y_val.shape[0], num_classes))])

X_te = torch.cat([X_te, torch.zeros(size=(num_nodes - X_te.shape[0], num_features))])
y_te = torch.cat([y_te, -torch.ones(size=(num_nodes - y_te.shape[0], num_classes))])

torch.manual_seed(41)

# get permutations
shuffle_tr, shuffle_val, shuffle_te = torch.randperm(num_nodes), torch.randperm(num_nodes), torch.randperm(num_nodes)
X_tr, X_val, X_te = X_tr[shuffle_tr], X_val[shuffle_val], X_te[shuffle_te]
y_tr, y_val, y_te = y_tr[shuffle_tr], y_val[shuffle_val], y_te[shuffle_te]
mask_tr, mask_val, mask_te = mask_tr[shuffle_tr], mask_val[shuffle_val], mask_te[shuffle_te]

In [8]:
lr = 1e-4
epochs = 300
print_every = 15

A = torch.sparse_coo_tensor(data.edge_index, torch.ones(data.edge_index.shape[1]))
model = TwoLayerGCN(A, num_classes, num_features, p=0.5).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_fn = nn.CrossEntropyLoss()

for t in range(1, epochs + 1):
    model.train()
    shuffle = torch.randperm(num_nodes)
    
    inputs, targets, shuffle_mask = X_tr[shuffle].to(device), y_tr[shuffle].to(device), mask_tr[shuffle]
    preds = model(inputs)
    train_loss = loss_fn(preds[shuffle_mask], targets[shuffle_mask])
    train_acc = (
        preds[shuffle_mask].argmax(axis=1) == targets[shuffle_mask].argmax(axis=1)
    ).float().mean()

    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    with torch.no_grad():
        model.eval()
        shuffle = torch.randperm(num_nodes)

        inputs, targets, shuffle_mask = X_val[shuffle].to(device), y_val[shuffle].to(device), mask_val[shuffle]
        preds = model(inputs)
        val_loss = loss_fn(preds[shuffle_mask], targets[shuffle_mask])
        val_acc = (
            preds[shuffle_mask].argmax(axis=1) == targets[shuffle_mask].argmax(axis=1)
        ).float().mean()

    if t % print_every == 0 or t == 1: print(
        'Epoch {:3d} | '.format(t) 
        + 'tr xe: {:.6f}; val xe: {:.6f}; '.format(train_loss.item(), val_loss.item())
        + 'tr acc: {:.4f}; val acc: {:.4f}'.format(train_acc.item(), val_acc.item())
    )
model.eval()
test_acc = (model(X_te)[mask_te].argmax(axis=1) == y_te[mask_te].argmax(axis=1)).float().mean()
print('test acc: {:4f}'.format(test_acc.item()))
# for some reason GCN takes 14 seconds to initialize on Cora

### Link Classification with GCN

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset = Planetoid('./data', data_name, transform=T.NormalizeFeatures())
data = dataset[0]

In [5]:
num_nodes, num_features = data.x.size()

In [6]:
X = data.x
A = torch.sparse_coo_tensor(data.edge_index, torch.ones(data.edge_index.shape[1])).to_dense()

# separate into train, val, test
X_tr, X_val, X_te = X[data.train_mask], X[data.val_mask], X[data.test_mask]
A_tr, A_val, A_te = A[data.train_mask], A[data.val_mask], A[data.test_mask]

# get indices of train, val, test
mask_tr = torch.cat([torch.ones(len(X_tr)), torch.zeros(num_nodes - len(X_tr))]).bool()
mask_val = torch.cat([torch.ones(len(X_val)), torch.zeros(num_nodes - len(X_val))]).bool()
mask_te = torch.cat([torch.ones(len(X_te)), torch.zeros(num_nodes - len(X_te))]).bool()

# fix shapes
X_tr = torch.cat([X_tr, torch.zeros(size=(num_nodes - X_tr.shape[0], num_features))], axis=0)
A_tr = torch.cat([A_tr, torch.zeros(size=(num_nodes - A_tr.shape[0], num_nodes))], axis=0)

X_val = torch.cat([X_val, torch.zeros(size=(num_nodes - X_val.shape[0], num_features))], axis=0)
A_val = torch.cat([A_val, torch.zeros(size=(num_nodes - A_val.shape[0], num_nodes))], axis=0)

X_te = torch.cat([X_te, torch.zeros(size=(num_nodes - X_te.shape[0], num_features))], axis=0)
A_te = torch.cat([A_te, torch.zeros(size=(num_nodes - A_te.shape[0], num_nodes))], axis=0)

# permute
shuffle_tr, shuffle_val, shuffle_te = torch.randperm(num_nodes), torch.randperm(num_nodes), torch.randperm(num_nodes)
X_tr, X_val, X_te = X_tr[shuffle_tr], X_val[shuffle_val], X_te[shuffle_te]
A_tr, A_val, A_te = A_tr[shuffle_tr], A_val[shuffle_val], A_te[shuffle_te]
mask_tr, mask_val, mask_te = mask_tr[shuffle_tr], mask_val[shuffle_val], mask_te[shuffle_te]

In [7]:
def report(targets, preds, mask):
    edge_rows, edge_cols = targets[mask].nonzero().t()
    tp = len(targets[mask].nonzero()[:, 0])
    fp = len((targets[mask][~edge_rows, ~edge_cols] != preds[mask][~edge_rows, ~edge_cols].round()).nonzero())
    fn = len((targets[mask][edge_rows, edge_cols] != preds[mask][edge_rows, edge_cols].round()).nonzero())
    prec, recall = tp / (tp + fp), tp / (tp + fn)
    return prec, recall

In [9]:
lr = 1e-7 # somehow 1e-12 works well
epochs = 500
print_every = 20

if data_name == 'Cora':
    model = TwoLayerGCNEdge(A_tr, num_features, num_features // 4, num_features // 16, p=0.5).to(device)
elif data_name == 'Pubmed':
    model = TwoLayerGCNEdge(A_tr, num_features, 64, 16, p=0).to(device)
print('model initialized.')
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_fn = nn.CrossEntropyLoss()

for t in range(1, epochs + 1):
    model.train()
    shuffle = torch.randperm(num_nodes)
    inputs, targets, shuffle_mask = X_tr[shuffle].to(device), A_tr[shuffle].to(device), mask_tr[shuffle]
    preds = model(inputs)
    train_loss = loss_fn(preds[shuffle_mask], targets[shuffle_mask])

    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    with torch.no_grad():
        model.eval()
        shuffle = torch.randperm(num_nodes)

        inputs, targets, shuffle_mask = X_tr[shuffle].to(device), A_tr[shuffle].to(device), mask_tr[shuffle]
        preds = model(inputs)
        train_prec, train_rcll = report(targets, preds, shuffle_mask)

        inputs, targets, shuffle_mask = X_val[shuffle].to(device), A_val[shuffle].to(device), mask_val[shuffle]
        preds = model(inputs)
        val_loss = loss_fn(preds[shuffle_mask], targets[shuffle_mask])
        val_prec, val_rcll = report(targets, preds, shuffle_mask)

    if t % print_every == 0 or t == 1: print(
        'Epoch {:3d} | '.format(t) 
        + 'tr xe: {:.4f}; val xe: {:.4f}; '.format(train_loss.item(), val_loss.item())
        + 'tr recall: {:.4f}; val recall: {:.4f}; '.format(train_rcll, val_rcll)
        + 'tr prec: {:.4f}; val prec: {:.4f}'.format(train_prec, val_prec)
    )
with torch.no_grad():
    model.eval()
    inputs, targets = X_te.to(device), A_te.to(device)
    preds = model(inputs)
    test_prec, test_rcll = report(targets, preds, mask_te)
    print('test recall: {:.4f}; test prec: {:.4f}'.format(test_rcll, test_prec))